# Bulk Workbook Copy on Tableau Server
To bulk copy Tableau workbooks from one project to another on a Tableau Server using the Tableau REST API.
This assumes you have access to Tableau Server (not Tableau Online) and necessary permissions.
Access to Tableau Server with admin or project leader rights.
Personal Access Token (PAT) or username/password

In [ ]:
#bash pip install tableauserverclient

## Authentication

In [ ]:
import tableauserverclient as TSC

# Replace with your actual Tableau Server info
SERVER_URL = '#'
SITE_ID = ''  # leave blank for default
TOKEN_NAME = 'aamverse'
TOKEN_SECRET = 'daaaammnnnngirl'

auth = TSC.PersonalAccessTokenAuth(token_name=TOKEN_NAME, personal_access_token=TOKEN_SECRET, site_id=SITE_ID)
server = TSC.Server(SERVER_URL, use_server_version=True)

##  Exe-cute

In [ ]:
with server.auth.sign_in(auth):
    all_projects, _ = server.projects.get()
    source_project = next((p for p in all_projects if p.name == 'Source Project Name'), None)
    target_project = next((p for p in all_projects if p.name == 'Target Project Name'), None)

    if not source_project or not target_project:
        raise Exception("Dude, your projects' name is not match just like me and her.")

    print(f"Source ID: {source_project.id}, Target ID: {target_project.id}")
    
    #Get Workbooks
    all_workbooks, _ = server.workbooks.get()
    workbooks_to_copy = [wb for wb in all_workbooks if wb.project_id == source_project.id]
    
    #Download each workbook
    for wb in workbooks_to_copy:
        print(f"Downloading workbook: {wb.name}")
        server.workbooks.download(wb.id, filepath=f"{wb.name}.twbx", include_extract=True)

    #Publish to new Project
    for wb in workbooks_to_copy:
        wb_file_path = f"{wb.name}.twbx"

        # Create new workbook item for upload
        new_wb = TSC.WorkbookItem(name=wb.name, project_id=target_project.id)
        print(f"Publishing workbook: {wb.name} to {target_project.name}")
        
        new_workbook = server.workbooks.publish(new_wb, wb_file_path, mode=TSC.Server.PublishMode.CreateNew)

### Clean up temp data

In [ ]:
import os

for wb in workbooks_to_copy:
    os.remove(f"{wb.name}.twbx")